In [31]:
import cv2
import numpy as np

def get_frame_with_vectors(video_path, frame_number, output_image_path):
    # Capture video from the file
    cap = cv2.VideoCapture(video_path)

    # Read the first frame
    ret, frame1 = cap.read()
    if not ret:
        print("Failed to read the video")
        return

    prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

    # Create a mask for visualizing the flow
    mask = np.zeros_like(frame1)

    frame_counter = 1
    while True:
        # Read the next frame
        ret, frame2 = cap.read()
        if not ret:
            print(f"Failed to read frame {frame_number}")
            break

        if frame_counter == frame_number:
            # Convert the current frame to grayscale
            gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

            # Calculate dense optical flow using Farneback method
            flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            
            # Compute the magnitude and angle of the flow vectors
            magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            
            # Set the hue based on the angle and normalize the magnitude
            mask[..., 0] = angle * 180 / np.pi / 2
            mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

            # Convert HSV to BGR
            flow_color = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
            
            # Draw motion vectors on the frame
            step = 16
            for y in range(0, frame2.shape[0], step):
                for x in range(0, frame2.shape[1], step):
                    dx, dy = flow[y, x]
                    # Change the color to yellow (BGR format: (0, 255, 255))
                    cv2.arrowedLine(frame2, (x, y), (int(x + dx), int(y + dy)), (0, 255, 255), 1, tipLength=0.5)
            # cv2.imshow('Dense Optical Flow with Motion Vectors', frame2)
            # Save the resulting frame as an image with high quality
            cv2.imwrite(output_image_path, frame2, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

            print(f"Frame {frame_number} saved as {output_image_path} with high quality")
            break

        # Convert the current frame to grayscale
        prev_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        frame_counter += 1

    # Release the capture
    cap.release()
    cv2.destroyAllWindows()

video_path = 'E:/FPP_files_and_codes/FPP_files/movies/Movie2.mp4'
# frame_number = 580
frame_number = 6650
output_image_path = 'E:/FPP_files_and_codes/Figures/Supplementary_Figure3/Motion_vector_in_image.jpg'
get_frame_with_vectors(video_path, frame_number, output_image_path)
